# ManTraNet Integration Plan for Sus Scrofa

**ManTra-Net: Manipulation Tracing Network For Detection and Localization of Image Forgeries**

This notebook documents the integration of ManTraNet into Sus Scrofa's forensic analysis pipeline.

## What is ManTraNet?

ManTraNet is a deep learning model that:
- **Detects** image manipulations (splicing, copy-move, removal, inpainting)
- **Localizes** manipulated regions by generating pixel-level forgery masks
- Outputs a heatmap where 0 = pristine, 1 = manipulated
- Pre-trained on 385 different manipulation types

## Integration Goals

1. ✅ Add ManTraNet as a detector in `ai_detection/detectors/`
2. ✅ Generate forgery masks and save to GridFS
3. ✅ Create audit findings for the auditor based on mask analysis
4. ✅ Add a new "Forgery Localization" tab in the UI
5. ✅ Display forgery masks overlaid on original images

## References

- **GitHub**: https://github.com/ISICV/ManTraNet
- **Colab Demo**: https://colab.research.google.com/drive/1ai4kVlI6w9rREqqYnTfpk3gM3YX9k-Ek
- **Paper**: CVPR 2019

## 2. ManTraNet Architecture Overview

Based on the Colab notebook, ManTraNet:

### Model Structure
- **Input**: RGB image, normalized to [-1, 1]
- **Architecture**: VGG-style feature extractor + manipulation tracer
- **Output**: Single-channel mask (H×W) with values 0-1
  - 0 = pristine pixel
  - 1 = manipulated pixel

### Inference Process

```python
# From Colab notebook
def decode_an_image_array(rgb, manTraNet, dn=1):
    # Normalize to [-1, 1]
    x = np.expand_dims(rgb.astype('float32')/255.*2-1, axis=0)
    
    # Downsample for speed (optional)
    x = x[:, ::dn, ::dn]
    
    # Run inference
    y = manTraNet.predict(x)[0, ..., 0]
    
    return y  # Shape: (H, W), values in [0, 1]
```

### Model Requirements
- TensorFlow 1.14 (original implementation)
- Python 3.6
- Pre-trained weights (~100MB)
- Separate virtual environment recommended

In [ ]:
"""
ManTraNet Detector — Image Manipulation Localization

Generates pixel-level forgery masks showing WHERE an image was manipulated.
Detects: copy-move, splicing, removal, inpainting, clone stamp, smudge tool.

Returns:
- Forgery mask (heatmap 0-1)
- Audit findings based on mask analysis
- Manipulated percentage and region count
"""

import logging
import numpy as np
from pathlib import Path
from typing import Optional
import subprocess
import json

from .base import BaseDetector, DetectionResult, DetectionMethod, ConfidenceLevel

logger = logging.getLogger(__name__)


class ManTraNetDetector(BaseDetector):
    """
    ManTraNet manipulation localization detector.
    
    Runs in subprocess using TF 1.14 environment (separate from main app).
    Generates forgery masks and creates audit findings based on analysis.
    """
    
    def get_order(self) -> int:
        """Run after AI detection but alongside OpenCV (order ~70)."""
        return 70
    
    def check_deps(self) -> bool:
        """Check if ManTraNet model and TF 1.14 environment exist."""
        try:
            mantranet_dir = Path(__file__).parent.parent / "ManTraNet"
            model_path = mantranet_dir / "pretrained_weights"
            
            if not model_path.exists():
                logger.warning("ManTraNet model not found - run: make mantranet-setup")
                return False
            
            return True
        except Exception as e:
            logger.error(f"Error checking ManTraNet deps: {e}")
            return False
    
    def detect(self, image_path: str, context=None) -> DetectionResult:
        """
        Generate forgery mask and analyze for manipulations.
        
        Returns DetectionResult with:
        - mask_id: GridFS ID of saved forgery mask
        - manipulated_percentage: % of pixels flagged
        - region_count: Number of discrete manipulated regions
        - audit_findings: Findings for the auditor
        """
        if not self.check_deps():
            return DetectionResult(
                method=DetectionMethod.ML_MODEL,
                is_ai_generated=None,
                confidence=ConfidenceLevel.NONE,
                score=0.0,
                evidence="ManTraNet not available - run: make mantranet-setup"
            )
        
        try:
            # Run inference via subprocess (TF 1.14 environment)
            mask, inference_time = self._run_inference(image_path)
            
            # Analyze mask
            analysis = self._analyze_mask(mask)
            
            # Save mask to GridFS
            mask_id = self._save_mask_to_gridfs(mask, context)
            
            # Create audit findings
            findings = self._create_audit_findings(analysis)
            
            # Determine overall verdict
            is_manipulated = analysis['manipulated_percentage'] > 5.0
            
            return DetectionResult(
                method=DetectionMethod.ML_MODEL,
                is_ai_generated=False,  # ManTraNet detects editing, not AI generation
                confidence=self._calculate_confidence(analysis),
                score=analysis['max_confidence'],
                evidence=self._format_evidence(analysis, inference_time),
                metadata={
                    'model': 'ManTraNet',
                    'mask_id': mask_id,
                    'manipulated_percentage': analysis['manipulated_percentage'],
                    'region_count': analysis['region_count'],
                    'max_confidence': analysis['max_confidence'],
                    'mean_confidence': analysis['mean_confidence'],
                    'inference_time_s': round(inference_time, 3),
                    'audit_findings': findings,
                }
            )
            
        except Exception as e:
            logger.error(f"ManTraNet error: {e}", exc_info=True)
            return DetectionResult(
                method=DetectionMethod.ML_MODEL,
                is_ai_generated=None,
                confidence=ConfidenceLevel.NONE,
                score=0.0,
                evidence=f"ManTraNet error: {e}"
            )

In [ ]:
def _analyze_mask(self, mask: np.ndarray) -> dict:
    """
    Analyze forgery mask to extract metrics.
    
    Returns:
        dict with manipulated_percentage, region_count, max_confidence,
        mean_confidence, and region_sizes
    """
    import cv2
    
    # Threshold mask at 0.5 for binary analysis
    binary_mask = (mask > 0.5).astype(np.uint8)
    
    # Calculate manipulated percentage
    total_pixels = mask.size
    manipulated_pixels = np.sum(binary_mask)
    manipulated_percentage = (manipulated_pixels / total_pixels) * 100
    
    # Find connected components (discrete manipulated regions)
    num_regions, labels, stats, centroids = cv2.connectedComponentsWithStats(
        binary_mask, connectivity=8
    )
    
    # Region 0 is background, so subtract 1
    region_count = num_regions - 1 if num_regions > 0 else 0
    
    # Get region sizes (excluding background)
    region_sizes = stats[1:, cv2.CC_STAT_AREA].tolist() if region_count > 0 else []
    
    # Confidence statistics (only from manipulated pixels)
    manipulated_values = mask[binary_mask == 1]
    max_confidence = float(np.max(manipulated_values)) if len(manipulated_values) > 0 else 0.0
    mean_confidence = float(np.mean(manipulated_values)) if len(manipulated_values) > 0 else 0.0
    
    return {
        'manipulated_percentage': manipulated_percentage,
        'region_count': region_count,
        'max_confidence': max_confidence,
        'mean_confidence': mean_confidence,
        'region_sizes': region_sizes,
    }


def _create_audit_findings(self, analysis: dict) -> list:
    """
    Create audit findings based on mask analysis.
    
    Returns list of findings following the plugin contract.
    """
    findings = []
    
    pct = analysis['manipulated_percentage']
    conf = analysis['mean_confidence']
    regions = analysis['region_count']
    
    # HIGH: >20% manipulated with high confidence
    if pct > 20 and conf > 0.7:
        findings.append({
            'level': 'HIGH',
            'category': 'Manipulation Localization',
            'description': f'Extensive manipulation detected: {pct:.1f}% of image affected in {regions} region(s)',
            'is_positive': False,
            'confidence': float(conf)
        })
    
    # MEDIUM: 5-20% manipulated or high % with lower confidence
    elif (pct > 5 and conf > 0.6) or (pct > 20 and conf > 0.5):
        findings.append({
            'level': 'MEDIUM',
            'category': 'Manipulation Localization',
            'description': f'Moderate manipulation detected: {pct:.1f}% of image affected in {regions} region(s)',
            'is_positive': False,
            'confidence': float(conf)
        })
    
    # LOW: 1-5% manipulated or borderline detection
    elif pct > 1:
        findings.append({
            'level': 'LOW',
            'category': 'Manipulation Localization',
            'description': f'Minor manipulation detected: {pct:.1f}% of image affected in {regions} region(s)',
            'is_positive': False,
            'confidence': float(conf)
        })
    
    # Positive finding: clean image
    elif pct < 0.5:
        findings.append({
            'level': 'MEDIUM',
            'category': 'Manipulation Localization',
            'description': 'No significant manipulation detected by forgery localization',
            'is_positive': True,
            'confidence': 1.0 - float(conf)
        })
    
    return findings

### Template

Create `templates/analyses/report/forgery.html`:

```html
{% extends "analyses/report/report_base.html" %}

{% block report_content %}
<div class="container-fluid">
    {% if error %}
        <div class="alert alert-warning">{{ error }}</div>
    {% elif mantranet.enabled %}
        <div class="row mb-4">
            <div class="col-12">
                <h3><i class="fa-solid fa-crosshairs me-2"></i>Manipulation Localization</h3>
                <p class="text-muted">
                    ManTraNet forgery detection highlights manipulated regions in red.
                    Darker red indicates higher confidence of manipulation.
                </p>
            </div>
        </div>
        
        <div class="row mb-4">
            <div class="col-md-6">
                <h5>Original Image</h5>
                <img src="{% url 'object_get' pk=analysis.image_id %}" 
                     class="img-fluid border" 
                     alt="Original">
            </div>
            <div class="col-md-6">
                <h5>Forgery Mask</h5>
                {% if mantranet.mask_id %}
                    <img src="{% url 'object_get' pk=mantranet.mask_id %}" 
                         class="img-fluid border" 
                         alt="Forgery Mask">
                {% else %}
                    <div class="alert alert-info">No forgery mask generated</div>
                {% endif %}
            </div>
        </div>
        
        <div class="row">
            <div class="col-12">
                <div class="card">
                    <div class="card-header"><strong>Detection Results</strong></div>
                    <div class="card-body">
                        <table class="table table-sm">
                            <tr>
                                <td><strong>Manipulated Area:</strong></td>
                                <td>{{ mantranet.manipulated_percentage|floatformat:2 }}%</td>
                            </tr>
                            <tr>
                                <td><strong>Number of Regions:</strong></td>
                                <td>{{ mantranet.region_count }}</td>
                            </tr>
                            <tr>
                                <td><strong>Max Confidence:</strong></td>
                                <td>{{ mantranet.max_confidence|floatformat:3 }}</td>
                            </tr>
                            <tr>
                                <td><strong>Mean Confidence:</strong></td>
                                <td>{{ mantranet.mean_confidence|floatformat:3 }}</td>
                            </tr>
                        </table>
                    </div>
                </div>
            </div>
        </div>
    {% endif %}
</div>
{% endblock %}
```

In [ ]:
# Example inference code from Colab
import numpy as np
import cv2
from datetime import datetime

def read_rgb_image(image_file):
    """Load image as RGB."""
    rgb = cv2.imread(image_file, 1)[..., ::-1]
    return rgb

def decode_an_image_array(rgb, manTraNet, dn=1):
    """
    Run ManTraNet inference on RGB image.
    
    Args:
        rgb: RGB numpy array
        manTraNet: Loaded model
        dn: Downsample factor (1=full res, 2=half res, etc.)
        
    Returns:
        mask: Forgery probability mask (H×W, values 0-1)
        time: Inference time
    """
    # Normalize to [-1, 1]
    x = np.expand_dims(rgb.astype('float32') / 255.0 * 2 - 1, axis=0)
    
    # Optional downsampling for speed
    x = x[:, ::dn, ::dn]
    
    # Inference
    t0 = datetime.now()
    y = manTraNet.predict(x)[0, ..., 0]
    t1 = datetime.now()
    
    return y, t1 - t0

def decode_an_image_file(image_file, manTraNet, dn=1):
    """Load image from file and run inference."""
    rgb = read_rgb_image(image_file)
    return decode_an_image_array(rgb, manTraNet, dn)


# Example usage:
# mask, inference_time = decode_an_image_file('suspicious.jpg', manTraNet, dn=1)
# print(f"Inference took {inference_time}")
# print(f"Mask shape: {mask.shape}, values range: [{mask.min():.3f}, {mask.max():.3f}]")

## 8. Example Audit Finding Scenarios

### Scenario 1: Major Manipulation

```python
# 25% of image manipulated, high confidence
analysis = {
    'manipulated_percentage': 25.3,
    'mean_confidence': 0.82,
    'region_count': 3
}

findings = [
    {
        'level': 'HIGH',  # 50 points subtracted
        'category': 'Manipulation Localization',
        'description': 'Extensive manipulation detected: 25.3% of image affected in 3 region(s)',
        'is_positive': False,
        'confidence': 0.82
    }
]

# Auditor impact: Base 50 - 50 (HIGH) = 0 (definitely fake)
```

### Scenario 2: Moderate Editing

```python
# 8% manipulated, moderate confidence
analysis = {
    'manipulated_percentage': 8.1,
    'mean_confidence': 0.65,
    'region_count': 1
}

findings = [
    {
        'level': 'MEDIUM',  # 15 points subtracted
        'category': 'Manipulation Localization',
        'description': 'Moderate manipulation detected: 8.1% of image affected in 1 region(s)',
        'is_positive': False,
        'confidence': 0.65
    }
]

# Auditor impact: Base 50 - 15 (MEDIUM) = 35 (likely fake)
```

### Scenario 3: Clean Image

```python
# <0.5% manipulated (essentially clean)
analysis = {
    'manipulated_percentage': 0.3,
    'mean_confidence': 0.15,
    'region_count': 0
}

findings = [
    {
        'level': 'MEDIUM',  # 15 points added
        'category': 'Manipulation Localization',
        'description': 'No significant manipulation detected by forgery localization',
        'is_positive': True,
        'confidence': 0.85
    }
]

# Auditor impact: Base 50 + 15 (MEDIUM positive) = 65 (likely real)
```

## Next Steps

1. ✅ Review this notebook for accuracy
2. ⏳ Create ManTraNetDetector plugin
3. ⏳ Setup TF 1.14 environment and download model
4. ⏳ Add UI tab and views
5. ⏳ Test on sample images
6. ⏳ Integrate with auditor scoring
7. ⏳ Document in README

## 7. Implementation Summary

### Files to Create

1. **`ai_detection/detectors/mantranet_detector.py`**
   - ManTraNetDetector class
   - Mask analysis logic
   - Audit findings generation
   - GridFS mask storage

2. **`ai_detection/mantranet_infer.py`**
   - Standalone inference script (TF 1.14 environment)
   - Called via subprocess from detector
   - Returns mask as JSON/NPY

3. **`templates/analyses/report/forgery.html`**
   - New tab template
   - Side-by-side original + mask display
   - Detection statistics table

4. **`ai_detection/Makefile`** updates
   - `make mantranet-setup`: Download model and setup TF 1.14 venv
   - `make mantranet-test`: Test inference

### Files to Modify

1. **`analyses/urls.py`**
   - Add `show_forgery` route

2. **`analyses/views.py`**
   - Add `show_forgery()` view function

3. **`templates/analyses/report/report_base.html`**
   - Add "Forgery Localization" tab to navigation

4. **`ai_detection/detectors/__init__.py`**
   - Import and register ManTraNetDetector

### Setup Commands

```bash
cd ai_detection/
make mantranet-setup   # Download ManTraNet repo and model
make mantranet-test    # Test inference on sample images
```

### Integration Flow

```
User uploads image
    ↓
Processing pipeline runs all plugins
    ↓
ManTraNetDetector.detect() called
    ↓
Runs inference via subprocess (TF 1.14 venv)
    ↓
Analyzes mask → creates audit_findings
    ↓
Saves mask to GridFS
    ↓
Auditor reads audit_findings
    ↓
Adds/subtracts points based on findings
    ↓
Final authenticity score calculated
    ↓
User views "Forgery Localization" tab
    ↓
Displays mask highlighting manipulated regions
```

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
import io

def mask_to_heatmap_image(mask, colormap='hot'):
    """
    Convert forgery mask to colored heatmap image.
    
    Args:
        mask: numpy array (H×W) with values 0-1
        colormap: matplotlib colormap name ('hot', 'jet', 'magma', etc.)
        
    Returns:
        PIL Image (RGB) of the heatmap
    """
    # Normalize to 0-255
    mask_normalized = (mask * 255).astype(np.uint8)
    
    # Apply colormap
    cmap = cm.get_cmap(colormap)
    colored = cmap(mask_normalized)
    
    # Convert to RGB (drop alpha channel)
    rgb = (colored[:, :, :3] * 255).astype(np.uint8)
    
    # Convert to PIL Image
    img = Image.fromarray(rgb)
    return img

def save_mask_to_bytes(mask, format='PNG'):
    """Save mask as image bytes for storage."""
    img = mask_to_heatmap_image(mask)
    buffer = io.BytesIO()
    img.save(buffer, format=format)
    buffer.seek(0)
    return buffer.getvalue()


# Example visualization
# from matplotlib import pyplot as plt
# 
# fig, axes = plt.subplots(1, 2, figsize=(12, 6))
# 
# # Original image
# axes[0].imshow(original_rgb)
# axes[0].set_title('Original Image')
# axes[0].axis('off')
# 
# # Forgery mask
# im = axes[1].imshow(mask, cmap='hot', vmin=0, vmax=1)
# axes[1].set_title('Forgery Mask (Red = Manipulated)')
# axes[1].axis('off')
# plt.colorbar(im, ax=axes[1])
# 
# plt.tight_layout()
# plt.show()

## 6. Mask Visualization

Convert the mask to a heatmap image for display:

## 5. Example ManTraNet Inference Code

Based on the Colab notebook, here's how ManTraNet runs inference:

### Update Tab Navigation

Add to `templates/analyses/report/report_base.html` navigation:

```html
{% if analysis.report.mantranet_localization.enabled %}
<li class="nav-item">
    <a class="nav-link {% if active_tab == 'forgery' %}active{% endif %}" 
       href="{% url 'show_forgery' analysis.id %}">
        <i class="fa-solid fa-crosshairs"></i> Forgery Localization
    </a>
</li>
{% endif %}
```

In [ ]:
@require_safe
@login_required
def show_forgery(request, analysis_id):
    """Shows forgery localization report (ManTraNet mask)."""
    context, error_response = _get_completed_analysis(request, analysis_id)
    if error_response:
        return error_response
    
    # Check if ManTraNet results exist
    mantranet_data = context['analysis'].report.get('mantranet_localization', {})
    
    if not mantranet_data.get('enabled'):
        context['error'] = 'Forgery localization not available for this image'
    
    context['active_tab'] = 'forgery'
    context['mantranet'] = mantranet_data
    
    return render(request, "analyses/report/forgery.html", context)

## 4. UI Integration — Forgery Localization Tab

Add a new tab to display the forgery mask:

### URL Route

Add to `analyses/urls.py`:
```python
re_path(r"^show/(?P<analysis_id>[\d]+)/forgery/$", views.show_forgery, name="show_forgery"),
```

### View Function

Add to `analyses/views.py`:

### Mask Analysis and Finding Creation

The detector analyzes the forgery mask to create audit findings:

## 3. Detector Plugin Implementation

Create `ai_detection/detectors/mantranet_detector.py`:

## 1. Plugin Contract Review

### Audit Findings Contract

Plugins communicate with the auditor via `audit_findings` in their results dictionary:

```python
results['plugin_name'] = {
    'audit_findings': [
        {
            'level': 'HIGH',  # LOW, MEDIUM, or HIGH
            'category': 'Manipulation Detection',
            'description': 'Copy-move forgery detected in 15.2% of image',
            'is_positive': False,  # False = evidence of fakery
            'confidence': 0.85  # Optional: 0.0-1.0
        }
    ],
    # ... other plugin-specific data
}
```

### Auditor Scoring System

- **Points**: LOW=5, MEDIUM=15, HIGH=50
- **Base Score**: 50 (uncertain)
- **Positive findings**: Add points (evidence of authenticity)
- **Negative findings**: Subtract points (evidence of manipulation)
- **Final Score**: Clamped to 0-100

### When to Use Each Level

- **HIGH**: Virtual certainty (>90% confidence) - e.g., >20% of image manipulated with high confidence
- **MEDIUM**: Strong evidence (70-90% confidence) - e.g., 5-20% manipulated or lower confidence
- **LOW**: Weak signal (50-70% confidence) - e.g., <5% manipulated or borderline detection